# Minicurso Processamento de Linguagem Natural - Prática 1

Autores:
* Fernando Sola Pereira
* Eduardo Soares de Paiva

In [234]:
##########################################
# libs python
##########################################
import os
import re
import warnings

##########################################
# libs externas
##########################################
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import common_texts
from IPython.display import display, HTML, Latex, Markdown
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import sent_tokenize
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE

##########################################
# configurações
##########################################
warnings.filterwarnings('ignore')
sns.set_style("whitegrid")
pd.options.display.max_rows = 2000
pd.options.display.max_colwidth = 200

# faz com que o nltk baixe arquivos e modelos necessários para este notebook
nltk.download('punkt')
nltk.download('stopwords')

##########################################
# variáveis globais
##########################################
stopwords = nltk.corpus.stopwords.words('portuguese')

# Local utilizado para armazenar arquivos de dados e checkpoints de modelos 
# (altere de acordo com a sua necessidade).
# Por padrão supõe-se que está sendo executado no google colab e que 
# o google drive do usuário está acessível.
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/sbsi/data'

# semente fixa para garantir reprodutibilidade
DEFAULT_RANDOM_SEED = 42

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [235]:
if not os.path.exists(DATA_PATH):
  try:
    from google.colab import drive
    drive.mount('/content/drive')

    os.makedirs(DATA_PATH)
  except:
    print('Não está executando no ambiente Google Colab!')
else:
  print('Diretório existente!')

Diretório existente!


In [236]:
# Dataset (B2WReview 2018)
# https://www.kaggle.com/datasets/fredericods/ptbr-sentiment-analysis-datasets
# Versão reduzida e balanceada
df_review = pd.read_csv('https://docs.google.com/uc?export=download&id=1_EKfnjomkWks4VqTMIpcEIb6nB5P0Xz2')
df_review.columns = ['label','text']
df_review['label'] = df_review['label'].apply(lambda x: 1 if x == 'positivo' else 0)
df_review.head()

,label,text
0,1,"A entrega foi no prazo, as americanas estão de parabéns. A smart tv é muito boa, a navegação na internete e pelos aplicativos e excelente, não trava, sem falar da imagem que é de surpreender. reco..."
1,1,"Excelente produto, por fora em material acrílico super resistente e por dentro em adamantio, faz milagre com qualquer bebida. Sugiro aproveitarem a promoção antes que acabe."
2,1,"produto mto bom, com essa garrafinha vc pode até servir água pro megazord. To pensando em vender minha tv pra comprar 1 garrafa dessa. RECOMENDO"
3,1,O barulho e minimo e o vento é bem forte na velocidade 2
4,0,MEU PRODUTO NAO FOI ENTREGUE E A AMERICANAS ESTA DESCONTANDO NA FATURA DO MEU CARTÃO


In [270]:
def remover_caracteres_especiais(t):
  # substituir números por tokens
  t = re.sub(pattern=r'\bR[$][ ]*[\d]+[\d,.]*\b', repl=r'<dinheiro>', string=t).strip()
  t = re.sub(pattern=r'\b\d+\b', repl=r'<numero>', string=t).strip()
  # substituir caracteres especiais
  t = re.sub(r'[!,.?()\[\]\n]', r' ', t)
  return t

corpus = [remover_caracteres_especiais(s).split() for d in df_review['text'] for s in sent_tokenize(d)]

model = Word2Vec(sentences=corpus, size=100, window=5, min_count=1, iter=10)
model.save(os.path.join(DATA_PATH, "word2vec.model"))

In [271]:
model.wv.most_similar(['ótimo'], topn=20)

[('excelente', 0.8155479431152344),
 ('bom', 0.781955897808075),
 ('otimo', 0.7528642416000366),
 ('perfeito', 0.7465569972991943),
 ('maravilhoso', 0.7304772734642029),
 ('Ótimo', 0.63785320520401),
 ('Excelente', 0.6109263896942139),
 ('incrível', 0.6026203632354736),
 ('fantástico', 0.6020241975784302),
 ('esperacular', 0.6009072065353394),
 ('lindo', 0.5778764486312866),
 ('razoável', 0.5547081232070923),
 ('ideal', 0.5448004603385925),
 ('bonito', 0.5435389876365662),
 ('justo', 0.5413516163825989),
 ('ótima', 0.5395393967628479),
 ('legal', 0.5392022132873535),
 ('adequado', 0.5285663604736328),
 ('bacana', 0.5206533670425415),
 ('espetacular', 0.5174404382705688)]

In [272]:
model.wv.most_similar(['péssimo'], topn=20)

[('horrível', 0.7915734052658081),
 ('pessimo', 0.7644460797309875),
 ('Péssimo', 0.7104436159133911),
 ('ruim', 0.6515201330184937),
 ('Pessimo', 0.5904371738433838),
 ('horrivel', 0.5884439945220947),
 ('umaruim', 0.518024742603302),
 ('eletrônico', 0.5124887824058533),
 ('falho', 0.5034551024436951),
 ('péssima', 0.5012702941894531),
 ('Americanas/Olist', 0.4991767108440399),
 ('eterna', 0.4783479571342468),
 ('fraco', 0.4767185151576996),
 ('falso', 0.4756103754043579),
 ('ineficaz', 0.4747752845287323),
 ('pior', 0.4629781246185303),
 ('compraxembalagemxentrega', 0.4576011896133423),
 ('ridiculo', 0.4520672559738159),
 ('Fino', 0.44809237122535706),
 ('Horrível', 0.44686129689216614)]

In [273]:
model.wv.similarity('ótimo', 'bom'), model.wv.similarity('ótimo', 'razoável'), model.wv.similarity('ótimo', 'ruim'), model.wv.similarity('ótimo', 'horrível')

(0.7819559, 0.5547082, 0.20638609, 0.22409406)

In [274]:
model.wv.most_similar(['caro'], topn=20)

[('barato', 0.8034005761146545),
 ('frete', 0.6255033016204834),
 ('<dinheiro>', 0.5943732261657715),
 ('reais', 0.5914076566696167),
 ('pagar', 0.5728238821029663),
 ('Paguei', 0.5550186634063721),
 ('pagamos', 0.5548049211502075),
 ('preço', 0.5452772974967957),
 ('grátis', 0.5293424129486084),
 ('paguei', 0.5284875631332397),
 ('ca4o', 0.5197276473045349),
 ('300reais', 0.514055609703064),
 ('compensa', 0.5131946206092834),
 ('custa', 0.5117173194885254),
 ('valor', 0.5114620923995972),
 ('cobram', 0.5073877573013306),
 ('suicídio', 0.4948153495788574),
 ('gratis', 0.48674172163009644),
 ('cara', 0.4862337112426758),
 ('Frete', 0.48114126920700073)]

In [339]:
palavras = ['bom', 'regular', 'ruim', 'ótimo', 'otimo', 'razoável', 'pior', 'melhor', 'péssimo', 'excelente', 'maravilhoso', 'perfeito', 'impecável', 'horrível', 'pessimo']

palavras += [p[0] for p in model.wv.most_similar(['ótimo'], topn=2)]
palavras += [p[0] for p in model.wv.most_similar(['bom'], topn=2)]
palavras += [p[0] for p in model.wv.most_similar(['razoável'], topn=2)]
palavras += [p[0] for p in model.wv.most_similar(['péssimo'], topn=2)]
palavras += [p[0] for p in model.wv.most_similar(['ruim'], topn=2)]

palavras += ['barato','caro', 'pesado', '<dinheiro>', 'reais', 'custa', 'paga']

palavras = set(palavras)
palavrasa = np.array([model.wv.word_vec(p.strip()) for p in palavras])

mms = MinMaxScaler()
v_palavras = mms.fit_transform(palavrasa)
pca = PCA(n_components=2, random_state=DEFAULT_RANDOM_SEED)
v_palavras = pca.fit_transform(v_palavras)


import plotly.express as px
fig = px.scatter(x=v_palavras[:, 0], y=v_palavras[:, 1], text=[f' {p}' for p in palavras], template='seaborn')
fig.update_traces(textposition='top center')
fig.update_layout(
    height=700,
    title_text='Palavras relacionadas a sentimento e valor'
)
fig.show()


# fig, ax = plt.subplots(figsize=(16,9), dpi=90)

# # pontos
# plt.plot(v_palavras[:, 0], v_palavras[:, 1], marker='o', linestyle='')

# # textos
# for x, y, s in zip(v_palavras[:, 0], v_palavras[:, 1], palavras):
#   plt.text(x, y, '  {}'.format(s), fontsize=10)

# plt.show()

In [340]:
conj_a = set([a[0] for a in model.wv.most_similar(positive=['ruim', 'caro'], negative=['bom'], topn=20)])

In [341]:
conj_b = set([a[0] for a in model.wv.most_similar(positive=['pior', 'caro'], negative=['melhor'], topn=20)])

In [344]:
conj_c = set([a[0] for a in model.wv.most_similar(positive=['ótimo', 'caro'], negative=['péssimo'], topn=20)])

In [345]:
conj_a & conj_b & conj_c 

{'barato'}

In [248]:
documents = [TaggedDocument(remover_caracteres_especiais(s).split(), [i]) for i, doc in enumerate(df_review['text']) for s in sent_tokenize(doc)]
model = Doc2Vec(documents, vector_size=100, window=5, min_count=10, workers=4, epochs=10, seed=DEFAULT_RANDOM_SEED)

In [249]:
vx = [model.infer_vector(d, steps=10) for d in df_review['text'].apply(lambda x: remover_caracteres_especiais(x).split())]

In [253]:
frase = 'loja com atendimento excelente e o produto nem se fala parabéns'
vfrase = model.infer_vector(frase.split(), steps=10)

sim = cosine_similarity(vx, [vfrase])

df_resultado = df_review.copy()
df_resultado['Similaridade'] = sim
df_resultado.sort_values('Similaridade', ascending=False).head()

,label,text,Similaridade
10201,0,Esta loja e uma porcaria nao recebi ainda o produto que amanha faz um mes.,0.819161
73176,1,Para quem gosta de porcaria este é o melhor aparelho que você pode ter.,0.811000
63001,1,Produto muito bom fácil de instalar e não precisa de configuração,0.807607
21116,1,Tv muito boa qualidade de imagem excelente som tambem muito bom produto de boa qualidade.,0.806876
22081,1,"Do jeito que eu imaginava,produto de excelente qualidade.",0.805874
